To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Long-Context GRPO for reinforcement learning — train stably at massive sequence lengths. Fine-tune models with up to 7x more context length efficiently. [Read Blog](https://unsloth.ai/docs/new/grpo-long-context)

3× faster training with optimized sequence packing — higher throughput with no quality loss.[Read Blog](https://unsloth.ai/docs/new/3x-faster-training-packing)

500k context-length fine-tuning — push long-context models further with memory-efficient training. [Read Blog](https://unsloth.ai/docs/new/500k-context-length-fine-tuning)

Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [2]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Mllama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 464b1386-40e3-4a8a-bafa-da9c11c2725a)')' thrown while requesting HEAD https://huggingface.co/unsloth/Llama-3.2-11B-Vision-Instruct-unsloth-bnb-4bit/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 10e0df6d-f367-4d09-bd20-65f836bfe356)')' thrown while requesting HEAD https://huggingface.co/unsloth/Llama-3.2-11B-Vision-Instruct-unsloth-bnb-4bit/resolve/main/model.safetensors
Retrying in 1s [Retry 1/5].


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5e122be1-00f8-488f-9177-4715a1a39b5c)')' thrown while requesting HEAD https://huggingface.co/unsloth/Llama-3.2-11B-Vision-Instruct-unsloth-bnb-4bit/resolve/main/processor_config.json
Retrying in 1s [Retry 1/5].


chat_template.json: 0.00B [00:00, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

**[NEW]** We also support finetuning ONLY the vision part of the model, or ONLY the language part. Or you can select both! You can also select to finetune the attention or the MLP layers!

In [3]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model.vision_model.transformer` require gradients


<a name="Data"></a>
### Data Prep
We'll be using a sampled version of the ROCO radiography dataset. You can access the dataset [here](https://huggingface.co/datasets/unsloth/Radiology_mini). The full dataset is [here](https://huggingface.co/datasets/eltorio/ROCOv2-radiology).

The dataset includes X-rays, CT scans and ultrasounds showcasing medical conditions and diseases. Each image has a caption written by experts describing it. The goal is to finetune a VLM to make it a useful analysis tool for medical professionals.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("unsloth/Radiology_mini", split="train")

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/79.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/327 [00:00<?, ? examples/s]

Let's take a look at the dataset, and check what the 1st example shows:

In [ ]:
dataset

In [ ]:
dataset[0]["image"]

In [ ]:
dataset[0]["caption"]

'Panoramic radiography shows an osteolytic lesion in the right posterior maxilla with resorption of the floor of the maxillary sinus (arrows).'

To format the dataset, all vision finetuning tasks should be formatted as follows:

```python
[
{ "role": "user",
  "content": [{"type": "text",  "text": instruction}, {"type": "image", "image": image} ]
},
{ "role": "assistant",
  "content": [{"type": "text",  "text": answer} ]
},
]
```

We will craft an custom instruction asking the VLM to be an expert radiographer. Notice also instead of just 1 instruction, you can add multiple turns to make it a dynamic conversation.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip drive/MyDrive/syn_dataset.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: syn_dataset/syn_data_2_annotated/syn_brochure_3038.json  
  inflating: __MACOSX/syn_dataset/syn_data_2_annotated/._syn_brochure_3038.json  
  inflating: syn_dataset/syn_data_2_annotated/syn_brochure_4657.json  
  inflating: __MACOSX/syn_dataset/syn_data_2_annotated/._syn_brochure_4657.json  
  inflating: syn_dataset/syn_data_2_annotated/syn_brochure_3192.json  
  inflating: __MACOSX/syn_dataset/syn_data_2_annotated/._syn_brochure_3192.json  
  inflating: syn_dataset/syn_data_2_annotated/syn_brochure_3468.json  
  inflating: __MACOSX/syn_dataset/syn_data_2_annotated/._syn_brochure_3468.json  
  inflating: syn_dataset/syn_data_2_annotated/syn_brochure_2629.json  
  inflating: __MACOSX/syn_dataset/syn_data_2_annotated/._syn_brochure_2629.json  
  inflating: syn_dataset/syn_data_2_annotated/syn_brochure_2783.json  
  inflating: __MACOSX/syn_dataset/syn_data_2_annotated/._syn_brochure_2783.json  
  inflating: syn_dataset/syn_data_2_annotated/syn_broch

In [13]:
!unzip drive/MyDrive/images_uniform.zip

unzip:  cannot find or open drive/MyDrive/images_uniform.zip, drive/MyDrive/images_uniform.zip.zip or drive/MyDrive/images_uniform.zip.ZIP.


In [ ]:
!unzip images_uniform.zip
import json
from pathlib import Path
from PIL import Image

def load_dataset_from_structure(root_path):
    dataset = []
    root = Path(root_path)

    for image_path in root.rglob('*.png'):

        image_folder = image_path.parent
        folder_name = image_folder.name

        if folder_name.endswith('_annotated'):
            continue

        json_folder_name = f"{folder_name}_annotated"
        json_filename = image_path.stem + ".json"

        json_path = image_folder.parent / json_folder_name / json_filename

        if json_path.exists():
            try:
                image_obj = Image.open(image_path).convert("RGB")

                with open(json_path, 'r', encoding='utf-8') as f:
                    annotation_data = json.load(f)

                text_content = json.dumps(annotation_data, ensure_ascii=False)

                dataset.append({
                    "image": image_obj,
                    "text": text_content
                })

                print(f"Loaded: {image_path.name}")

            except Exception as e:
                print(f"Error when reading the file: {image_path.name}: {e}")
        else:
            print(f"No corresponding file for {json_path}")

    return dataset

root_directory = "images_uniform"

data = load_dataset_from_structure(root_directory)

print(f"\nTotally {len(data)} data")

In [ ]:
# Load syn_data
root_directory = "syn_dataset"

data_syn = load_dataset_from_structure(root_directory)

print(f"\nTotally {len(data)} data")

In [15]:
instruction = """You are analyzing a supermarket brochure page in German.

Extract ALL product deals from this image. For each product, identify:

1. **Product Name**: Full product name including brand
2. **Price**: The main selling price (in euros, format: "X.XX")
3. **Discount**: Discount percentage if shown (format: "XX" without %)
4. **Unit**: Product size/quantity (e.g., "je 500-g-Pckg.", "je 1-l-Becher", "je 750 g")
5. **Original Price**: Original price before discount if shown
6. **bbox**: Bounding box of the product region in normalized coordinates
   Format: `[x_min, y_min, x_max, y_max]`
   - All values must be between **0 and 1**
   - Coordinates refer to the **entire brochure page**, not cropped regions
   - `x_min < x_max`, `y_min < y_max`
   - Bounding box should tightly cover the product’s visual area (product card, price box, text, etc.)

IMPORTANT RULES:
- Extract information from EACH visible product/deal on the page
- Group information by product card/region (don't mix products)
- Only extract text that is clearly visible and readable
- For prices, include ONLY the numeric value (e.g., "17.99" not "€17.99")
- For discounts, include ONLY the number (e.g., "20" not "-20%")
- If information is not visible or unclear, use null
- Pay special attention to product cards, promotional boxes, and price tags

Return ONLY a JSON array with this EXACT structure:
[
  {{
    "product_name": "Brand ProductName",
    "price": "X.XX",
    "discount": "XX" or null,
    "unit": "je XXX g/ml/l/kg-XXX" or null,
    "original_price": "X.XX" or null,
    "bbox": [0.0, 0.0, 1.0, 1.0]
  }}
]

Example output:
[
  {{
    "product_name": "Baileys Irish Cream",
    "price": "17.99",
    "discount": null,
    "unit": "je 700-ml-Becher",
    "original_price": null,
    "bbox": [0.12, 0.30, 0.28, 0.71]
  }},
  {{
    "product_name": "Landliebe Joghurt",
    "price": "1.49",
    "discount": "20",
    "unit": "je 500-g-Dose",
    "original_price": "1.99",
    "bbox": [0.42, 0.18, 0.57, 0.55]
  }}
]

Return ONLY the JSON array, no other text or explanation."""

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["text"]} ]
        },
    ]
    return { "messages" : conversation }
pass

Let's convert the dataset into the "correct" format for finetuning:

In [16]:
converted_dataset = [convert_to_conversation(sample) for sample in data]
converted_dataset_syn = [convert_to_conversation(sample) for sample in data_syn]

The first example is now structured like below:

In [ ]:
converted_dataset[0]

In [ ]:
converted_dataset_syn[0]

Before we do any finetuning, maybe the vision model already knows how to analyse the images? Let's check if this is the case!

In [ ]:
data[0]["image"]

In [ ]:
data_syn[0]["image"]

In [47]:
FastVisionModel.for_inference(model) # Enable for inference!

image = data[0]["image"]
instruction = """You are analyzing a supermarket brochure page in German.

Extract ALL product deals from this image. For each product, identify:

1. **Product Name**: Full product name including brand
2. **Price**: The main selling price (in euros, format: "X.XX")
3. **Discount**: Discount percentage if shown (format: "XX" without %)
4. **Unit**: Product size/quantity (e.g., "je 500-g-Pckg.", "je 1-l-Becher", "je 750 g")
5. **Original Price**: Original price before discount if shown
6. **bbox**: Bounding box of the product region in normalized coordinates
   Format: `[x_min, y_min, x_max, y_max]`
   - All values must be between **0 and 1**
   - Coordinates refer to the **entire brochure page**, not cropped regions
   - `x_min < x_max`, `y_min < y_max`
   - Bounding box should tightly cover the product’s visual area (product card, price box, text, etc.)

IMPORTANT RULES:
- Extract information from EACH visible product/deal on the page
- Group information by product card/region (don't mix products)
- Only extract text that is clearly visible and readable
- For prices, include ONLY the numeric value (e.g., "17.99" not "€17.99")
- For discounts, include ONLY the number (e.g., "20" not "-20%")
- If information is not visible or unclear, use null
- Pay special attention to product cards, promotional boxes, and price tags

Return ONLY a JSON array with this EXACT structure:
[
  {{
    "product_name": "Brand ProductName",
    "price": "X.XX",
    "discount": "XX" or null,
    "unit": "je XXX g/ml/l/kg-XXX" or null,
    "original_price": "X.XX" or null,
    "bbox": [0.0, 0.0, 1.0, 1.0]
  }}
]

Example output:
[
  {{
    "product_name": "Baileys Irish Cream",
    "price": "17.99",
    "discount": null,
    "unit": "je 700-ml-Becher",
    "original_price": null,
    "bbox": [0.12, 0.30, 0.28, 0.71]
  }},
  {{
    "product_name": "Landliebe Joghurt",
    "price": "1.49",
    "discount": "20",
    "unit": "je 500-g-Dose",
    "original_price": "1.99",
    "bbox": [0.42, 0.18, 0.57, 0.55]
  }}
]

Return ONLY the JSON array, no other text or explanation."""

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 0.1, min_p = 0.1)

[{"product_name": "Emmentaler", "price": "1.79", "discount": null, "unit": "je 400 g Packung", "original_price": null, "bbox": [0.033999999999999996, 0.07100000000000001, 0.4849999999999999, 0.2729999999999999]}, {"product_name": "Premium Eiscreme", "price": "2.49", "discount": null, "unit": "je 500 g Becher", "original_price": null, "bbox": [0.033999999999999996, 0.2879999999999999, 0.4849999999999999, 0.4859999999999999]}, {"product_name": "Rohschinken", "price": "0.99", "discount": null, "unit": "je 200 g Packung", "original_price": null, "bbox": [0.5039999999999999, 0.07100000000000001, 0.9999999999999999, 0.2729999999999999]}, {"product_name": "Doppelkeks", "price": "14.99", "discount": null, "unit": "je 175 g Beutel", "original_price": null, "bbox": [0.033999999999999996, 0.4879999999999999, 0.4849999999999999, 0.7129999999999999]}, {"product_name": "Goldbären", "price": "14.99", "discount": null, "unit": "je 150 g Beutel", "original_price": null, "bbox": [0.5039999999999999, 0.4

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

We use our new `UnslothVisionDataCollator` which will help in our vision finetuning setup.

In [50]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
import random

FastVisionModel.for_training(model) # Enable for training!

# mix_dataset = converted_dataset[:173] * 4 + converted_dataset[:69] + converted_dataset_syn[:1151]
# random.seed(3407)
# random.shuffle(mix_dataset)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_dataset[:19],
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 1,
        warmup_steps = 1,
        num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_llama_20",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!

image = data[174]["image"]
instruction = """You are analyzing a supermarket brochure page in German.

Extract ALL product deals from this image. For each product, identify:

1. **Product Name**: Full product name including brand
2. **Price**: The main selling price (in euros, format: "X.XX")
3. **Discount**: Discount percentage if shown (format: "XX" without %)
4. **Unit**: Product size/quantity (e.g., "je 500-g-Pckg.", "je 1-l-Becher", "je 750 g")
5. **Original Price**: Original price before discount if shown
6. **bbox**: Bounding box of the product region in normalized coordinates
   Format: `[x_min, y_min, x_max, y_max]`
   - All values must be between **0 and 1**
   - Coordinates refer to the **entire brochure page**, not cropped regions
   - `x_min < x_max`, `y_min < y_max`
   - Bounding box should tightly cover the product’s visual area (product card, price box, text, etc.)

IMPORTANT RULES:
- Extract information from EACH visible product/deal on the page
- Group information by product card/region (don't mix products)
- Only extract text that is clearly visible and readable
- For prices, include ONLY the numeric value (e.g., "17.99" not "€17.99")
- For discounts, include ONLY the number (e.g., "20" not "-20%")
- If information is not visible or unclear, use null
- Pay special attention to product cards, promotional boxes, and price tags

Return ONLY a JSON array with this EXACT structure:
[
  {{
    "product_name": "Brand ProductName",
    "price": "X.XX",
    "discount": "XX" or null,
    "unit": "je XXX g/ml/l/kg-XXX" or null,
    "original_price": "X.XX" or null,
    "bbox": [0.0, 0.0, 1.0, 1.0]
  }}
]

Example output:
[
  {{
    "product_name": "Baileys Irish Cream",
    "price": "17.99",
    "discount": null,
    "unit": "je 700-ml-Becher",
    "original_price": null,
    "bbox": [0.12, 0.30, 0.28, 0.71]
  }},
  {{
    "product_name": "Landliebe Joghurt",
    "price": "1.49",
    "discount": "20",
    "unit": "je 500-g-Dose",
    "original_price": "1.99",
    "bbox": [0.42, 0.18, 0.57, 0.55]
  }}
]

Return ONLY the JSON array, no other text or explanation."""

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 0.1, min_p = 0.1)

In [ ]:
data[174]["text"]

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [44]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

[]

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
data[0]

### Evaluation

In [ ]:
from tqdm import tqdm
if True:
    from unsloth import FastVisionModel
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name = "/content/outputs_llama_20/checkpoint-10", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True, # Set to False for 16bit LoRA
    )
    FastVisionModel.for_inference(model) # Enable for inference!

BATCH_SIZE = 8

image = data[0]["image"]
instruction = """You are analyzing a supermarket brochure page in German.

Extract ALL product deals from this image. For each product, identify:

1. **Product Name**: Full product name including brand
2. **Price**: The main selling price (in euros, format: "X.XX")
3. **Discount**: Discount percentage if shown (format: "XX" without %)
4. **Unit**: Product size/quantity (e.g., "je 500-g-Pckg.", "je 1-l-Becher", "je 750 g")
5. **Original Price**: Original price before discount if shown
6. **bbox**: Bounding box of the product region in normalized coordinates
   Format: `[x_min, y_min, x_max, y_max]`
   - All values must be between **0 and 1**
   - Coordinates refer to the **entire brochure page**, not cropped regions
   - `x_min < x_max`, `y_min < y_max`
   - Bounding box should tightly cover the product’s visual area (product card, price box, text, etc.)

IMPORTANT RULES:
- Extract information from EACH visible product/deal on the page
- Group information by product card/region (don't mix products)
- Only extract text that is clearly visible and readable
- For prices, include ONLY the numeric value (e.g., "17.99" not "€17.99")
- For discounts, include ONLY the number (e.g., "20" not "-20%")
- If information is not visible or unclear, use null
- Pay special attention to product cards, promotional boxes, and price tags

Return ONLY a JSON array with this EXACT structure:
[
  {{
    "product_name": "Brand ProductName",
    "price": "X.XX",
    "discount": "XX" or null,
    "unit": "je XXX g/ml/l/kg-XXX" or null,
    "original_price": "X.XX" or null,
    "bbox": [0.0, 0.0, 1.0, 1.0]
  }}
]

Example output:
[
  {{
    "product_name": "Baileys Irish Cream",
    "price": "17.99",
    "discount": null,
    "unit": "je 700-ml-Becher",
    "original_price": null,
    "bbox": [0.12, 0.30, 0.28, 0.71]
  }},
  {{
    "product_name": "Landliebe Joghurt",
    "price": "1.49",
    "discount": "20",
    "unit": "je 500-g-Dose",
    "original_price": "1.99",
    "bbox": [0.42, 0.18, 0.57, 0.55]
  }}
]

Return ONLY the JSON array, no other text or explanation."""

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

all_generated_texts = []
print(f"Batch Size = {BATCH_SIZE}...")

eval_data = data[174:]

for i in tqdm(range(0, len(eval_data), BATCH_SIZE)):

    batch_data = eval_data[i : i + BATCH_SIZE]

    batch_images = [[item["image"]] for item in batch_data]
    print(batch_images)
    batch_prompts = []

    for item in batch_data:
        messages = [
            {"role": "user", "content": [
                {"type": "image"},
                {"type": "text", "text": instruction}
            ]}
        ]

        text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False
        )
        batch_prompts.append(text)

    try:
        inputs = tokenizer(
            images=batch_images,
            text=batch_prompts,
            add_special_tokens=False,
            return_tensors="pt",
            padding=True,
        ).to("cuda")
    except ValueError as e:
        raise e

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=2048,
            use_cache=True,
            temperature=0.1,
            min_p=0.1
        )

    input_len = inputs.input_ids.shape[1]
    new_tokens = generated_ids[:, input_len:]

    batch_outputs = tokenizer.batch_decode(new_tokens, skip_special_tokens=True)
    print(batch_outputs)

    all_generated_texts.extend(batch_outputs)

print(f"Inference finished, totally {len(all_generated_texts)} results.")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)

==((====))==  Unsloth 2026.1.4: Fast Mllama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

开始 Batch 推理，Batch Size = 8...


  0%|          | 0/6 [00:00<?, ?it/s]

[[<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC712568D0>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC71257140>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC71256B40>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC712572F0>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC712573E0>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC71256ED0>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC712575C0>], [<PIL.Image.Image image mode=RGB size=1024x1448 at 0x7FDC712571A0>]]


...

['[{"product_name": "COVO Jaffa Cake", "price": "1.99", "discount": null, "unit": "je 300 g", "original_price": null, "bbox": [0.12121212121212147, 0.036999999999999985, 0.4989999999999996, 0.2729999999999999]}, {"product_name": "COVO Mini Doppelkeks", "price": "1.49", "discount": null, "unit": "je 6 x 37,5 g", "original_price": null, "bbox": [0.5219999999999996, 0.036999999999999985, 0.8769999999999992, 0.2729999999999999]}, {"product_name": "COVO Schwedisches Hafergebäck Schoko", "price": "3.99", "discount": null, "unit": "je 525 g", "original_price": null, "bbox": [0.12121212121212147, 0.3019999999999999, 0.4989999999999996, 0.5219999999999999]}, {"product_name": "COVO Schoko & Keks Sticks Milchschokolade oder Zartbiter-schokolade", "price": "1.49", "discount": null, "unit": "je 90 g", "original_price": null, "bbox": [0.5219999999999996, 0.2819999999999999, 0.8769999999999992, 0.5219999999999999]}, {"product_name": "COVO Waffel Gebäck Mischung", "price": "2.89", "discount": null, "u

 83%|████████▎ | 5/6 [19:51<03:39, 219.30s/it]

['[{"product_name": "DOUCEUR Weihnachtsmann Kekse mit Karamellgeschmack", "price": "1.19", "discount": null, "unit": "je 120 g", "original_price": null, "bbox": [0.12121212121212147, 0.3529999999999999, 0.5019999999999996, 0.7129999999999999]}, {"product_name": "DOUCEUR Schokogeld Milchschokolade", "price": "1.99", "discount": null, "unit": "je 150 g", "original_price": null, "bbox": [0.5219999999999996, 0.3529999999999999, 0.8879999999999996, 0.7129999999999999]}, {"product_name": "DOUCEUR Edel-Marzipan-Stollen", "price": "4.99", "discount": null, "unit": "je 1.000 g", "original_price": null, "bbox": [0.12121212121212147, 0.7289999999999999, 0.8769999999999996, 0.9639999999999999]}]', '[{"product_name": "DOUCEUR Baumkuchensplitzen", "price": "2.99", "discount": null, "unit": "je 150 g", "original_price": null, "bbox": [0.12121212121212144, 0.2727272727272728, 0.5012121212121215, 0.48242424242424244]}, {"product_name": "DOUCEUR Gewürzspekulatius", "price": "1.69", "discount": null, "un

100%|██████████| 6/6 [22:00<00:00, 220.12s/it]

['[{"product_name": "Weihnachts-Deko", "price": "7.99", "discount": null, "unit": null, "original_price": null, "bbox": [0.12121212121212147, 0.0, 0.9047619047619049, 0.48549723756906044]}, {"product_name": "LED-Stern-Lichterkette", "price": "1.99", "discount": null, "unit": null, "original_price": null, "bbox": [0.12121212121212147, 0.3708563535911604, 0.49005555555555554, 0.48549723756906044]}, {"product_name": "Tafelkerzen", "price": "2.99", "discount": null, "unit": "je Set", "original_price": null, "bbox": [0.12121212121212147, 0.5214475134475135, 0.3611111111111111, 0.7304475134475135]}, {"product_name": "LED-Ech-wachskerzen", "price": "9.99", "discount": null, "unit": "je Set", "original_price": null, "bbox": [0.3611111111111111, 0.5214475134475135, 0.6219797979797979, 0.7304475134475135]}, {"product_name": "LED-Deko-Baume-Set", "price": "4.99", "discount": null, "unit": "Ca. 1,8 cm und 18 cm hoch je Set", "original_price": null, "bbox": [0.6219797979797979, 0.5214475134475135, 

In [56]:
import json
with open("/content/drive/MyDrive/all_generated_texts_syn_llama_20.json", "w", encoding="utf-8") as f:
    json.dump(all_generated_texts, f, ensure_ascii=False, indent=2)

In [ ]:
import json
import re
from difflib import SequenceMatcher
import numpy as np

def calculate_iou(box1, box2):
    if not box1 or not box2: return 0.0

    try:
        b1 = [float(x) for x in box1]
        b2 = [float(x) for x in box2]
    except:
        return 0.0

    x_left   = max(b1[0], b2[0])
    y_top    = max(b1[1], b2[1])
    x_right  = min(b1[2], b2[2])
    y_bottom = min(b1[3], b2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    b1_area = (b1[2] - b1[0]) * (b1[3] - b1[1])
    b2_area = (b2[2] - b2[0]) * (b2[3] - b2[1])

    union_area = b1_area + b2_area - intersection_area

    if union_area <= 0: return 0.0

    return intersection_area / union_area

def clean_json_output(text):
    text = re.sub(r'```json\s*', '', text)
    text = re.sub(r'```', '', text)
    match = re.search(r'(\[.*\]|\{.*\})', text, re.DOTALL)
    if match: json_str = match.group(0)
    else: json_str = text
    try:
        data = json.loads(json_str)
        if isinstance(data, dict): return [data]
        return data
    except: return None

def normalize_price(price_str):
    if price_str is None: return ""
    return str(price_str).replace("€", "").replace("$", "").replace(",", ".").strip()

def evaluate(predictions, dataset, name_threshold=0.5, iou_threshold=0.5):
    stats = {
        "total_gt_items": 0,
        "total_pred_items": 0,
        "correct_matches": 0,
        "price_correct": 0,
        "bbox_correct": 0,
        "bbox_iou_sum": 0.0,
        "json_parse_errors": 0
    }

    bad_cases = []

    print(f"Evaluating {len(predictions)} data...")

    for i, (pred_text, item) in enumerate(zip(predictions, dataset)):


        raw_gt = item.get('text')
        if raw_gt is None: gt_list = []
        else: gt_list = raw_gt

        if isinstance(gt_list, str):
            try: gt_list = json.loads(gt_list)
            except: gt_list = []
        if isinstance(gt_list, dict): gt_list = [gt_list]
        if not isinstance(gt_list, list): gt_list = []

        pred_list = clean_json_output(pred_text)
        if pred_list is None:
            stats["json_parse_errors"] += 1
            stats["total_gt_items"] += len(gt_list)
            continue

        stats["total_gt_items"] += len(gt_list)
        stats["total_pred_items"] += len(pred_list)

        matched_gt_indices = set()

        for p_item in pred_list:
            p_name = str(p_item.get("name", "")).lower()
            p_price = normalize_price(p_item.get("price", ""))
            p_bbox = p_item.get("bbox", p_item.get("box", []))

            best_score = 0
            best_gt_idx = -1

            for gt_idx, gt_item in enumerate(gt_list):
                if gt_idx in matched_gt_indices: continue

                g_name = str(gt_item.get("name", "")).lower()
                score = SequenceMatcher(None, p_name, g_name).ratio()

                if score > best_score:
                    best_score = score
                    best_gt_idx = gt_idx

            if best_score > name_threshold:
                stats["correct_matches"] += 1
                matched_gt_indices.add(best_gt_idx)

                gt_match = gt_list[best_gt_idx]

                g_price = normalize_price(gt_match.get("price", ""))
                if p_price == g_price:
                    stats["price_correct"] += 1

                g_bbox = gt_match.get("bbox", gt_match.get("box", []))

                iou = calculate_iou(p_bbox, g_bbox)
                stats["bbox_iou_sum"] += iou

                if iou > iou_threshold:
                    stats["bbox_correct"] += 1
                else:
                    bad_cases.append({
                        "id": i,
                        "type": "Bad BBox",
                        "name": p_name,
                        "iou": f"{iou:.2f}",
                        "pred_box": p_bbox,
                        "gt_box": g_bbox
                    })

    tp = stats["correct_matches"]
    fp = stats["total_pred_items"] - tp
    fn = stats["total_gt_items"] - tp

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    price_acc = stats["price_correct"] / tp if tp > 0 else 0
    bbox_acc = stats["bbox_correct"] / tp if tp > 0 else 0
    mean_iou = stats["bbox_iou_sum"] / tp if tp > 0 else 0

    results = {
        "F1 Score": f"{f1:.2%}",
        "Precision": f"{precision:.2%}",
        "Recall": f"{recall:.2%}",
        "Price Accuracy": f"{price_acc:.2%}",
        "BBox Accuracy": f"{bbox_acc:.2%}",   # IoU > 0.5
        "Mean IoU": f"{mean_iou:.2f}",
        "Count (TP/GT)": f"{tp} / {stats['total_gt_items']}"
    }

    return results, bad_cases


metrics, bad_cases = evaluate(all_generated_texts, data[174:])

print("-" * 30)
print("report")
print("-" * 30)
for k, v in metrics.items():
    print(f"{k:<20}: {v}")
print("-" * 30)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Select ONLY 1 to save! (Both not needed!)

# Save locally to 16bit
if False: model.save_pretrained_merged("unsloth_finetune", tokenizer,)

# To export and save to your Hugging Face account
if False: model.push_to_hub_merged("YOUR_USERNAME/unsloth_finetune", tokenizer, token = "PUT_HERE")

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme)
</div>
